# Import Libraries


In [1]:
import os
import shutil

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

from utils import create_model, augment_images, flatten_datasets, fine_tune
from config import config, models

# Prepare Training Data

Our data will be split into 80% training, and holding out 20% for testing later on. The training set we will be used for 5-fold cross validation during model fitting, so our training set will be further split into training and validation.


In [2]:
data_dir = "dataset/2-cropped"

train_set, test_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    seed=config["seed_value"],
    image_size=config["img_shape"],
    batch_size=config["batch_size"],
    label_mode="categorical",
    subset="both",
)

train_images, train_labels = flatten_datasets(train_set)

Found 447 files belonging to 2 classes.
Using 358 files for training.
Using 89 files for validation.


# Defining the Base Model

This project will be using MobileNetV3-Small (1) and EfficientNetV2-B0 (2). We can easily get this from the TensorFlow API. Let's set a `flag` variable so we can easily set which model we want to test with.


In [3]:
flag = 3
base_model = models[flag]["base_model"]
model_name = models[flag]["model_name"]
num_layers_to_freeze = models[flag]["num_layers_to_freeze"]

In [4]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_3
1 rescaling_2
2 Conv
3 Conv/BatchNorm
4 tf.__operators__.add_27
5 re_lu_32
6 tf.math.multiply_27
7 multiply_18
8 expanded_conv/depthwise
9 expanded_conv/depthwise/BatchNorm
10 re_lu_33
11 expanded_conv/project
12 expanded_conv/project/BatchNorm
13 expanded_conv/Add
14 expanded_conv_1/expand
15 expanded_conv_1/expand/BatchNorm
16 re_lu_34
17 expanded_conv_1/depthwise/pad
18 expanded_conv_1/depthwise
19 expanded_conv_1/depthwise/BatchNorm
20 re_lu_35
21 expanded_conv_1/project
22 expanded_conv_1/project/BatchNorm
23 expanded_conv_2/expand
24 expanded_conv_2/expand/BatchNorm
25 re_lu_36
26 expanded_conv_2/depthwise
27 expanded_conv_2/depthwise/BatchNorm
28 re_lu_37
29 expanded_conv_2/project
30 expanded_conv_2/project/BatchNorm
31 expanded_conv_2/Add
32 expanded_conv_3/expand
33 expanded_conv_3/expand/BatchNorm
34 re_lu_38
35 expanded_conv_3/depthwise/pad
36 expanded_conv_3/depthwise
37 expanded_conv_3/depthwise/BatchNorm
38 re_lu_39
39 expanded_conv_3/squeeze_excite/AvgPool
40 

# Fitting the Model

We define our cross validation strategy as `StratifiedKFold` and set the splits to 5 to ensure that the distribution of image samples per split remains equal all throughout to counter class imbalance.


In [5]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=config["seed_value"])

# Loop over the dataset to create separate folds
for i, (train_idx, valid_idx) in enumerate(
    cv.split(train_images, np.argmax(train_labels, axis=1))
):
    print(f"Fold {i + 1}")

    # Create a new model instance
    model = fine_tune(base_model, config, num_layers_to_freeze)

    # Get the training and validation data
    X_train, y_train = train_images[train_idx], train_labels[train_idx]
    X_valid, y_valid = train_images[valid_idx], train_labels[valid_idx]

    # Augment ONLY training data
    X_train, y_train = augment_images(X_train, y_train, 5)

    # Compute weights
    weights = compute_class_weight(
        class_weight="balanced", classes=np.unique([0, 1]), y=y_train.argmax(axis=1)
    )
    weights = dict(zip(np.unique([0, 1]), weights))

    # Define checkpoint path and checkpoint callback
    path = f"checkpoints/{model_name}/fold_{i+1}"
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

    # Save the weights using the `checkpoint_path` format
    checkpoint_path = path + "/cp-{epoch:04d}.ckpt"
    model.save_weights(checkpoint_path.format(epoch=0))

    # Define filename for CSV log
    filename = f"results/{model_name}/fold_{i+1}.csv"

    # Define callbacks
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            verbose=1,
        ),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10),
        tf.keras.callbacks.CSVLogger(filename, separator=",", append=False),
    ]

    # Fit the model on the train set and evaluate on the validation set
    model.fit(
        X_train,
        y_train,
        batch_size=config["batch_size"],
        epochs=config["epochs"],
        class_weight=weights,
        validation_data=(X_valid, y_valid),
        verbose=,
        callbacks=callbacks,
    )

Fold 1
Epoch 1/200

Epoch 1: val_loss improved from inf to 0.80579, saving model to checkpoints/efficientnetv2-b0/fold_1/cp-0001.ckpt
27/27 - 55s - loss: 0.8522 - precision: 0.4551 - recall: 0.4551 - f1_score: 0.5709 - false_negatives: 935.0000 - true_positives: 781.0000 - false_positives: 935.0000 - true_negatives: 781.0000 - val_loss: 0.8058 - val_precision: 0.3889 - val_recall: 0.3889 - val_f1_score: 0.5098 - val_false_negatives: 44.0000 - val_true_positives: 28.0000 - val_false_positives: 44.0000 - val_true_negatives: 28.0000 - 55s/epoch - 2s/step
Epoch 2/200

Epoch 2: val_loss improved from 0.80579 to 0.78828, saving model to checkpoints/efficientnetv2-b0/fold_1/cp-0002.ckpt
27/27 - 6s - loss: 0.8044 - precision: 0.4930 - recall: 0.4930 - f1_score: 0.6048 - false_negatives: 870.0000 - true_positives: 846.0000 - false_positives: 870.0000 - true_negatives: 846.0000 - val_loss: 0.7883 - val_precision: 0.3889 - val_recall: 0.3889 - val_f1_score: 0.5098 - val_false_negatives: 44.0000 -